In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gzip

# Part 1: Fragment Length Frequency and Normalization
# Read the BED file
def read_bed_file(bed_file):
    with gzip.open(bed_file, 'rt') as f:
        df = pd.read_csv(f, sep='\t', header=None, usecols=[1, 2])
    df.columns = ['start', 'end']
    return df

# Calculate fragment lengths
def calculate_fragment_lengths(df):
    return (df['end'] - df['start']).abs()

# Compute normalized frequency
def compute_normalized_frequency(lengths):
    hist, bins = np.histogram(lengths, bins=100, range=(0, 1000))  # Adjust range as needed
    total = np.sum(hist)
    normalized_freq = hist / total
    bin_centers = (bins[:-1] + bins[1:]) / 2
    return bin_centers, normalized_freq

# Plot normalized frequency
def plot_normalized_frequency(bin_centers, normalized_freq, title, output_file):
    plt.figure(figsize=(8, 6))
    plt.plot(bin_centers, normalized_freq, 'b-', label='Query Normalized Frequency')
    plt.xlabel('Fragment Length (base pairs)')
    plt.ylabel('Normalized Frequency [A.U.]')
    plt.title(title)
    plt.grid(True)
    plt.legend()
    plt.savefig(output_file)
    plt.close()

# Part 2: Rescaling
# Read reference histogram
def read_reference_histogram(ref_file):
    ref_df = pd.read_csv(ref_file, sep='\t', header=None)
    ref_df.columns = ['length', 'frequency']
    return ref_df

# Rescale query data to match reference
def rescale_query(lengths, ref_df):
    # Create histogram of query lengths
    query_hist, bins = np.histogram(lengths, bins=100, range=(0, 1000))
    total_query = np.sum(query_hist)
    query_freq = query_hist / total_query
    
    # Interpolate reference frequencies to match query bins
    ref_lengths = ref_df['length'].values
    ref_freq = ref_df['frequency'].values
    bin_centers = (bins[:-1] + bins[1:]) / 2
    interp_ref_freq = np.interp(bin_centers, ref_lengths, ref_freq)
    
    # Calculate scaling factors
    scaling_factors = np.where(query_freq > 0, interp_ref_freq / query_freq, 0)
    
    # Subsample lengths
    rescaled_lengths = []
    for length in lengths:
        bin_idx = np.digitize(length, bins) - 1
        if bin_idx < len(scaling_factors) and np.random.random() < scaling_factors[bin_idx]:
            rescaled_lengths.append(length)
    
    return np.array(rescaled_lengths)

# Main execution
def main():
    # File paths (update these paths as needed)
    bed_file = 'query.bed.gz'
    ref_file = 'reference.hist'
    
    # Part 1: Fragment length frequency
    df = read_bed_file(bed_file)
    lengths = calculate_fragment_lengths(df)
    bin_centers, normalized_freq = compute_normalized_frequency(lengths)
    
    # Plot original normalized frequency
    plot_normalized_frequency(bin_centers, normalized_freq, 
                            'Query Fragment Length Distribution', 
                            'query_normalized.png')
    
    # Part 2: Rescaling
    ref_df = read_reference_histogram(ref_file)
    rescaled_lengths = rescale_query(lengths, ref_df)
    
    # Compute normalized frequency for rescaled data
    rescaled_bin_centers, rescaled_freq = compute_normalized_frequency(rescaled_lengths)
    
    # Plot comparison
    plt.figure(figsize=(8, 6))
    plt.plot(bin_centers, normalized_freq, 'b-', label='Original Query')
    plt.plot(rescaled_bin_centers, rescaled_freq, 'r--', label='Rescaled Query')
    plt.plot(ref_df['length'], ref_df['frequency'], 'g-', label='Reference')
    plt.xlabel('Fragment Length (base pairs)')
    plt.ylabel('Normalized Frequency [A.U.]')
    plt.title('Fragment Length Distribution Comparison')
    plt.grid(True)
    plt.legend()
    plt.savefig('distribution_comparison.png')
    plt.close()

if __name__ == '__main__':
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'query.bed.gz'